# Creation of minke whale annotations from extracted audio clips


## Purpose of this notebook
This notebook describes the steps involved in automatically creating minke whale annotations from small audio clips extracted with Raven. 

The start and end times of the annotations are considered to be the start and end of teh audio clip. The min and max frequency of the annotations are 0 Hz and the Nyquist frequency, respectively.

## Deployment folders

The data provided were separated into folders corresponding to different deployments. A deployment_info.csv file was created in each of these folders and contains the metadata for each deployment.

![noise_scotland_folders.png](img/noise_scotland_folders.png)


## Import libraries and define functions used throughout

In [58]:
from ecosound.core.annotation import Annotation
from ecosound.core.metadata import DeploymentInfo
from ecosound.core.audiotools import Sound
from ecosound.core.tools import filename_to_datetime
import ecosound.core.decorators
import os
import re
import pandas as pd
import numpy as np
import uuid
from datetime import datetime

@ecosound.core.decorators.listinput
def filename_to_datetime2(files):
    pattern ={"string_pattern":"_[0-9]{6}_[0-9]{6}","time_format":"_%y%m%d_%H%M%S"}            
    p = re.compile(pattern["string_pattern"])
    
    timestamps = [None] * len(files)
    for idx, file in enumerate(files):
        datestr = p.search(file)
        timestamps[idx] = datetime.strptime(datestr[0], pattern["time_format"])               
    return timestamps

@ecosound.core.decorators.listinput
def filename_to_datetime3(files):
    pattern ={"string_pattern":"_[0-9]{8}_[0-9]{6}","time_format":"_%Y%m%d_%H%M%S"}            
    p = re.compile(pattern["string_pattern"])
    
    timestamps = [None] * len(files)
    for idx, file in enumerate(files):
        datestr = p.search(file)
        timestamps[idx] = datetime.strptime(datestr[0], pattern["time_format"])               
    return timestamps

@ecosound.core.decorators.listinput
def filename_to_datetime4(files):
    #_2021-05-08_20-00-04
    pattern ={"string_pattern":"_[0-9]{4}-[0-9]{2}-[0-9]{2}_[0-9]{2}-[0-9]{2}-[0-9]{2}","time_format":"_%Y-%m-%d_%H-%M-%S"}            
    p = re.compile(pattern["string_pattern"])
    timestamps = [None] * len(files)
    for idx, file in enumerate(files):
        datestr = p.search(file)
        timestamps[idx] = datetime.strptime(datestr[0], pattern["time_format"])               
    return timestamps

@ecosound.core.decorators.listinput
def filename_to_datetime5(files):
    #Depl987_1347451_190611004441_minke_1_0
    pattern ={"string_pattern":"_[0-9]{12}_","time_format":"_%y%m%d%H%M%S_"}            
    p = re.compile(pattern["string_pattern"])
    timestamps = [None] * len(files)
    for idx, file in enumerate(files):
        datestr = p.search(file)
        timestamps[idx] = datetime.strptime(datestr[0], pattern["time_format"])               
    return timestamps

def create_minke_annot(audio_dir, deployment_file, file_ext, label_class, label_subclass):
    files_list = os.listdir(audio_dir)
    annot_stack = []
    for file in files_list:
        if file.endswith(file_ext):
            print(file)
            # retrieve file start date and time
            try:
                file_timestamp = filename_to_datetime(file)
            except:
                try:
                    file_timestamp = filename_to_datetime2(file)
                except:
                    try:
                        file_timestamp = filename_to_datetime3(file)
                    except:
                        try:
                            file_timestamp = filename_to_datetime4(file)
                        except:
                            file_timestamp = filename_to_datetime5(file)                                
            

            # retrieve file duration
            audio = Sound(os.path.join(audio_dir, file))
            file_dur = audio.file_duration_sec

            # define annotations start times (relative to start begining of the audio file)
            t1 = [0]
            t2 = [file_dur]
            # create the annotatiom object
            annot = Annotation()

            annot.data['time_min_offset'] = t1
            annot.data['time_max_offset'] = t2
            annot.insert_values(audio_file_start_date=file_timestamp[0])
            annot.data['time_min_date'] = pd.to_datetime(
                annot.data['audio_file_start_date'] + pd.to_timedelta(
                    annot.data['time_min_offset'], unit='s'))
            annot.data['time_max_date'] = pd.to_datetime(
                annot.data['audio_file_start_date'] +
                pd.to_timedelta(annot.data['time_max_offset'], unit='s'))
            annot.insert_values(audio_channel=1)
            annot.insert_values(audio_file_name=os.path.splitext(os.path.basename(file))[0])
            annot.insert_values(audio_file_dir=audio_dir)
            annot.insert_values(audio_file_extension=os.path.splitext(file)[1])
            annot.insert_values(frequency_min=0)
            annot.insert_values(software_version=0)
            annot.insert_values(operator_name='xavier')
            annot.insert_values(entry_date=datetime.now())
            annot.insert_values(frequency_max=audio.file_sampling_frequency/2)
            annot.insert_values(label_class=label_class)
            annot.insert_values(label_subclass=label_subclass)
            annot.insert_values(from_detector=False)
            annot.insert_values(software_name='custom_python')
            annot.data['uuid'] = annot.data.apply(lambda _: str(uuid.uuid4()), axis=1)
            annot.data['duration'] = annot.data['time_max_offset'] - annot.data['time_min_offset']        
            # add metadata            
            annot.insert_metadata(os.path.join(audio_dir, deployment_file)) 
            # stack annotatiosn for each file
            annot_stack.append(annot)
            # check that evrything looks fine
            annot.check_integrity(verbose=False, ignore_frequency_duplicates=True)

    # concatenate all annotations
    annot_concat = annot_stack[0]
    for an_idx in range(1, len(annot_stack)):
        annot_concat = annot_concat + annot_stack[an_idx]
    annot_concat.check_integrity(verbose=False, ignore_frequency_duplicates=True)
    return annot_concat

### Dataset 1: UK-MSS-MorayFirth-201806_274-Helmsdale

Definition of all the paths of all folders with the raw annotation and audio files for this deployment.

In [22]:
audio_dir = r'D:\NOAA\2022_Minke_whale_detector\manual_annotations\UK-MSS-MorayFirth-201806_274-Helmsdale'
deployment_file = r'deployment_info.csv' 
file_ext = 'wav'

label_class = 'MW'  # label to use for the sound clip
label_subclass = '' # label to use for the sound clip subclass

In [23]:
annot = create_minke_annot(audio_dir, deployment_file, file_ext, label_class, label_subclass)

Depl274_Helmsdale_180612_210000_minke_2_1.wav
Depl274_Helmsdale_180602_080000_minke_1_0.wav
Depl274_Helmsdale_180602_233000_minke_2_0.wav
Depl274_Helmsdale_180602_233000_minke_2_1.wav
Depl274_Helmsdale_180603_003000_minke_2_0.wav
Depl274_Helmsdale_180603_003000_minke_2_1.wav
Depl274_Helmsdale_180609_193000_minke_1_1.wav
Depl274_Helmsdale_180609_193000_minke_1_2.wav
Depl274_Helmsdale_180609_193000_minke_2_0.wav
Depl274_Helmsdale_180611_010000_minke_2_0.wav
Depl274_Helmsdale_180611_010000_minke_2_1.wav
Depl274_Helmsdale_180611_043000_minke_2_0.wav
Depl274_Helmsdale_180612_173000_minke_2_0.wav
Depl274_Helmsdale_180612_180000_minke_2_0.wav
Depl274_Helmsdale_180612_180000_minke_2_1.wav
Depl274_Helmsdale_180612_180000_minke_2_2.wav
Depl274_Helmsdale_180612_180000_minke_2_3.wav
Depl274_Helmsdale_180612_210000_minke_2_0.wav


Let's look at the summary of annotations that were created:

In [24]:
annot.summary()

label_class,MW,Total
deployment_ID,,
UK-MSS-MorayFirth-201806_274-Helmsdale,18,18
Total,18,18


The dataset can now be saved as a Raven annotation file and netcdf4 file:

In [25]:
annot.to_netcdf(os.path.join(audio_dir, 'Annotations_dataset_' + annot.data['deployment_ID'][0] +' annotations.nc'))
annot.to_raven(audio_dir, outfile='Annotations_dataset_' + annot.data['deployment_ID'][0] +'.Table.1.selections.txt', single_file=True)

### Dataset 2: UK-MSS-MorayFirth-201904_360-Helmsdale

Definition of all the paths of all folders with the raw annotation and audio files for this deployment.

In [26]:
audio_dir = r'D:\NOAA\2022_Minke_whale_detector\manual_annotations\UK-MSS-MorayFirth-201904_360-Helmsdale'
deployment_file = r'deployment_info.csv' 
file_ext = 'wav'

label_class = 'MW'  # label to use for the sound clip
label_subclass = '' # label to use for the sound clip subclass

In [27]:
annot = create_minke_annot(audio_dir, deployment_file, file_ext, label_class, label_subclass)

Depl360_Helmsdale_190720_000000_minke_1_0.wav
Depl360_Helmsdale_190602_213000_minke_2_0.wav
Depl360_Helmsdale_190602_213000_minke_2_1.wav
Depl360_Helmsdale_190602_223000_minke_2_0.wav
Depl360_Helmsdale_190602_223000_minke_2_1.wav
Depl360_Helmsdale_190602_223000_minke_2_2.wav
Depl360_Helmsdale_190602_223000_minke_2_3.wav
Depl360_Helmsdale_190602_223000_minke_2_4.wav
Depl360_Helmsdale_190602_230000_minke_2_0.wav
Depl360_Helmsdale_190602_230000_minke_2_1.wav
Depl360_Helmsdale_190602_230000_minke_2_2.wav
Depl360_Helmsdale_190602_230000_minke_2_3.wav
Depl360_Helmsdale_190602_230000_minke_2_4.wav
Depl360_Helmsdale_190602_233000_minke_2_0.wav
Depl360_Helmsdale_190609_003000_minke_2_0.wav
Depl360_Helmsdale_190616_210000_minke_2_0.wav
Depl360_Helmsdale_190616_210000_minke_2_1.wav
Depl360_Helmsdale_190616_210000_minke_2_2.wav
Depl360_Helmsdale_190617_070000_minke_1_0.wav
Depl360_Helmsdale_190617_073000_minke_2_0.wav
Depl360_Helmsdale_190617_073000_minke_2_1.wav
Depl360_Helmsdale_190617_073000_mi

Let's look at the summary of annotations that were created:

In [28]:
annot.summary()

label_class,MW,Total
deployment_ID,,
UK-MSS-MorayFirth-201904_360-Helmsdale,54,54
Total,54,54


The dataset can now be saved as a Raven annotation file and netcdf4 file:

In [29]:
annot.to_netcdf(os.path.join(audio_dir, 'Annotations_dataset_' + annot.data['deployment_ID'][0] +' annotations.nc'))
annot.to_raven(audio_dir, outfile='Annotations_dataset_' + annot.data['deployment_ID'][0] +'.Table.1.selections.txt', single_file=True)

### Dataset 3: UK-MSS-MorayFirth-201908_421-Helmsdale

Definition of all the paths of all folders with the raw annotation and audio files for this deployment.

In [30]:
audio_dir = r'D:\NOAA\2022_Minke_whale_detector\manual_annotations\UK-MSS-MorayFirth-201908_421-Helmsdale'
deployment_file = r'deployment_info.csv' 
file_ext = 'wav'

label_class = 'MW'  # label to use for the sound clip
label_subclass = '' # label to use for the sound clip subclass

In [31]:
annot = create_minke_annot(audio_dir, deployment_file, file_ext, label_class, label_subclass)

Depl421_Helmsdale_191028_231000_minke_2_0.wav
Depl421_Helmsdale_190907_204000_minke_1_0.wav
Depl421_Helmsdale_190907_204000_minke_1_1.wav
Depl421_Helmsdale_190907_211000_minke_1_0.wav
Depl421_Helmsdale_190907_211000_minke_2_1.wav
Depl421_Helmsdale_190907_211000_minke_2_2.wav
Depl421_Helmsdale_190908_014000_minke_2_0.wav
Depl421_Helmsdale_190908_064000_minke_1_0.wav
Depl421_Helmsdale_190908_064000_minke_2_1.wav
Depl421_Helmsdale_190908_091000_minke_1_0.wav
Depl421_Helmsdale_190910_084000_minke_2_0.wav
Depl421_Helmsdale_190916_221000_minke_1_0.wav
Depl421_Helmsdale_190918_194000_minke_1_0.wav
Depl421_Helmsdale_190918_201000_minke_1_0.wav
Depl421_Helmsdale_190918_201000_minke_1_1.wav
Depl421_Helmsdale_190918_201000_minke_1_2.wav
Depl421_Helmsdale_190918_201000_minke_1_3.wav
Depl421_Helmsdale_190918_201000_minke_1_4.wav
Depl421_Helmsdale_190918_211000_minke_1_0.wav
Depl421_Helmsdale_190919_194000_minke_2_0.wav
Depl421_Helmsdale_190919_194000_minke_2_1.wav
Depl421_Helmsdale_190919_194000_mi

Depl421_Helmsdale_191003_214000_minke_1_2.wav
Depl421_Helmsdale_191004_024000_minke_1_2.wav
Depl421_Helmsdale_191004_024000_minke_1_3.wav
Depl421_Helmsdale_191004_024000_minke_1_4.wav
Depl421_Helmsdale_191004_024000_minke_1_5.wav
Depl421_Helmsdale_191004_024000_minke_2_0.wav
Depl421_Helmsdale_191004_024000_minke_2_1.wav
Depl421_Helmsdale_191004_024000_minke_2_6.wav
Depl421_Helmsdale_191004_024000_minke_2_7.wav
Depl421_Helmsdale_191004_024000_minke_2_8.wav
Depl421_Helmsdale_191004_024000_minke_2_9.wav
Depl421_Helmsdale_191004_024000_minke_2_10.wav
Depl421_Helmsdale_191004_031000_minke_1_0.wav
Depl421_Helmsdale_191004_034000_minke_1_0.wav
Depl421_Helmsdale_191004_034000_minke_1_1.wav
Depl421_Helmsdale_191004_034000_minke_1_2.wav
Depl421_Helmsdale_191004_034000_minke_1_3.wav
Depl421_Helmsdale_191004_034000_minke_1_4.wav
Depl421_Helmsdale_191004_034000_minke_1_6.wav
Depl421_Helmsdale_191004_034000_minke_1_8.wav
Depl421_Helmsdale_191004_034000_minke_2_5.wav
Depl421_Helmsdale_191004_034000_m

Depl421_Helmsdale_191016_224000_minke_2_3.wav
Depl421_Helmsdale_191016_224000_minke_2_4.wav
Depl421_Helmsdale_191016_224000_minke_2_5.wav
Depl421_Helmsdale_191016_231000_minke_1_0.wav
Depl421_Helmsdale_191016_231000_minke_2_1.wav
Depl421_Helmsdale_191018_131000_minke_1_0.wav
Depl421_Helmsdale_191018_201000_minke_1_0.wav
Depl421_Helmsdale_191018_204000_minke_1_0.wav
Depl421_Helmsdale_191018_211000_minke_1_0.wav
Depl421_Helmsdale_191019_214000_minke_1_0.wav
Depl421_Helmsdale_191019_214000_minke_2_1.wav
Depl421_Helmsdale_191020_191000_minke_2_0.wav
Depl421_Helmsdale_191020_191000_minke_2_1.wav
Depl421_Helmsdale_191020_191000_minke_2_2.wav
Depl421_Helmsdale_191020_191000_minke_2_3.wav
Depl421_Helmsdale_191020_194000_minke_1_6.wav
Depl421_Helmsdale_191020_194000_minke_1_7.wav
Depl421_Helmsdale_191020_194000_minke_1_9.wav
Depl421_Helmsdale_191020_194000_minke_1_10.wav
Depl421_Helmsdale_191020_194000_minke_1_11.wav
Depl421_Helmsdale_191020_194000_minke_1_12.wav
Depl421_Helmsdale_191020_194000

Let's look at the summary of annotations that were created:

In [32]:
annot.summary()

label_class,MW,Total
deployment_ID,,
UK-MSS-MorayFirth-201908_421-Helmsdale,453,453
Total,453,453


The dataset can now be saved as a Raven annotation file and netcdf4 file:

In [33]:
annot.to_netcdf(os.path.join(audio_dir, 'Annotations_dataset_' + annot.data['deployment_ID'][0] +' annotations.nc'))
annot.to_raven(audio_dir, outfile='Annotations_dataset_' + annot.data['deployment_ID'][0] +'.Table.1.selections.txt', single_file=True)

### Dataset 4: UK-SAMS-WestScotland-201711-StantonBank

Definition of all the paths of all folders with the raw annotation and audio files for this deployment.

In [34]:
audio_dir = r'D:\NOAA\2022_Minke_whale_detector\manual_annotations\UK-SAMS-WestScotland-201711-StantonBank'
deployment_file = r'deployment_info.csv' 
file_ext = 'wav'

label_class = 'MW'  # label to use for the sound clip
label_subclass = '' # label to use for the sound clip subclass

In [35]:
annot = create_minke_annot(audio_dir, deployment_file, file_ext, label_class, label_subclass)

PAM_20180312_200004_000_minke_0.wav
PAM_20180312_200004_000_minke_1.wav
PAM_20180312_200004_000_minke_2.wav
PAM_20180315_040004_000_minke_0.wav
PAM_20180321_010004_000_minke_0.wav
PAM_20180321_010004_000_minke_1.wav
PAM_20180321_010004_000_minke_2.wav
PAM_20180321_010004_000_minke_3.wav
PAM_20180321_010004_000_minke_4.wav
PAM_20180321_010004_000_minke_5.wav
PAM_20180321_010004_000_minke_6.wav
PAM_20180321_010004_000_minke_7.wav
PAM_20180321_020004_000_minke_0.wav
PAM_20180321_020004_000_minke_1.wav
PAM_20180321_020004_000_minke_10.wav
PAM_20180321_020004_000_minke_11.wav
PAM_20180321_020004_000_minke_12.wav
PAM_20180321_020004_000_minke_13.wav
PAM_20180321_020004_000_minke_2.wav
PAM_20180321_020004_000_minke_3.wav
PAM_20180321_020004_000_minke_4.wav
PAM_20180321_020004_000_minke_5.wav
PAM_20180321_020004_000_minke_6.wav
PAM_20180321_020004_000_minke_7.wav
PAM_20180321_020004_000_minke_8.wav
PAM_20180321_020004_000_minke_9.wav
PAM_20180321_030004_000_minke_0.wav
PAM_20180321_030004_000_

PAM_20180322_220004_000_minke_0.wav
PAM_20180322_220004_000_minke_1.wav
PAM_20180322_220004_000_minke_10.wav
PAM_20180322_220004_000_minke_11.wav
PAM_20180322_220004_000_minke_12.wav
PAM_20180322_220004_000_minke_13.wav
PAM_20180322_220004_000_minke_14.wav
PAM_20180322_220004_000_minke_15.wav
PAM_20180322_220004_000_minke_16.wav
PAM_20180322_220004_000_minke_17.wav
PAM_20180322_220004_000_minke_18.wav
PAM_20180322_220004_000_minke_19.wav
PAM_20180322_220004_000_minke_2.wav
PAM_20180322_220004_000_minke_20.wav
PAM_20180322_220004_000_minke_21.wav
PAM_20180322_220004_000_minke_22.wav
PAM_20180322_220004_000_minke_23.wav
PAM_20180322_220004_000_minke_24.wav
PAM_20180322_220004_000_minke_25.wav
PAM_20180322_220004_000_minke_3.wav
PAM_20180322_220004_000_minke_4.wav
PAM_20180322_220004_000_minke_5.wav
PAM_20180322_220004_000_minke_6.wav
PAM_20180322_220004_000_minke_7.wav
PAM_20180322_220004_000_minke_8.wav
PAM_20180322_220004_000_minke_9.wav
PAM_20180322_230004_000_minke_0.wav
PAM_20180322

PAM_20180705_105741_000_minke_2.wav
PAM_20180705_105741_000_minke_3.wav
PAM_20180705_105741_000_minke_4.wav
PAM_20180705_105741_000_minke_5.wav
PAM_20180705_105741_000_minke_6.wav
PAM_20180705_105741_000_minke_7.wav
PAM_20180705_105741_000_minke_8.wav
PAM_20180705_105741_000_minke_9.wav
PAM_20180705_115741_000_minke_0.wav
PAM_20180705_115741_000_minke_1.wav
PAM_20180705_115741_000_minke_10.wav
PAM_20180705_115741_000_minke_11.wav
PAM_20180705_115741_000_minke_12.wav
PAM_20180705_115741_000_minke_13.wav
PAM_20180705_115741_000_minke_14.wav
PAM_20180705_115741_000_minke_15.wav
PAM_20180705_115741_000_minke_16.wav
PAM_20180705_115741_000_minke_17.wav
PAM_20180705_115741_000_minke_18.wav
PAM_20180705_115741_000_minke_19.wav
PAM_20180705_115741_000_minke_2.wav
PAM_20180705_115741_000_minke_20.wav
PAM_20180705_115741_000_minke_21.wav
PAM_20180705_115741_000_minke_3.wav
PAM_20180705_115741_000_minke_4.wav
PAM_20180705_115741_000_minke_5.wav
PAM_20180705_115741_000_minke_6.wav
PAM_20180705_115

PAM_20180814_185741_000_minke_16.wav
PAM_20180814_185741_000_minke_17.wav
PAM_20180814_185741_000_minke_18.wav
PAM_20180814_185741_000_minke_19.wav
PAM_20180814_185741_000_minke_2.wav
PAM_20180814_185741_000_minke_20.wav
PAM_20180814_185741_000_minke_21.wav
PAM_20180814_185741_000_minke_22.wav
PAM_20180814_185741_000_minke_23.wav
PAM_20180814_185741_000_minke_24.wav
PAM_20180814_185741_000_minke_25.wav
PAM_20180814_185741_000_minke_26.wav
PAM_20180814_185741_000_minke_27.wav
PAM_20180814_185741_000_minke_28.wav
PAM_20180814_185741_000_minke_29.wav
PAM_20180814_185741_000_minke_3.wav
PAM_20180814_185741_000_minke_30.wav
PAM_20180814_185741_000_minke_31.wav
PAM_20180814_185741_000_minke_32.wav
PAM_20180814_185741_000_minke_33.wav
PAM_20180814_185741_000_minke_34.wav
PAM_20180814_185741_000_minke_35.wav
PAM_20180814_185741_000_minke_36.wav
PAM_20180814_185741_000_minke_37.wav
PAM_20180814_185741_000_minke_38.wav
PAM_20180814_185741_000_minke_39.wav
PAM_20180814_185741_000_minke_4.wav
PAM_

Let's look at the summary of annotations that were created:

In [36]:
annot.summary()

label_class,MW,Total
deployment_ID,,
UK-SAMS-WestScotland-201711-StantonBank,762,762
Total,762,762


The dataset can now be saved as a Raven annotation file and netcdf4 file:

In [37]:
annot.to_netcdf(os.path.join(audio_dir, 'Annotations_dataset_' + annot.data['deployment_ID'][0] +' annotations.nc'))
annot.to_raven(audio_dir, outfile='Annotations_dataset_' + annot.data['deployment_ID'][0] +'.Table.1.selections.txt', single_file=True)

### Dataset 5: UK-SAMS-WestScotland-202009-N1

Definition of all the paths of all folders with the raw annotation and audio files for this deployment.

In [38]:
audio_dir = r'D:\NOAA\2022_Minke_whale_detector\manual_annotations\UK-SAMS-WestScotland-202009-N1'
deployment_file = r'deployment_info.csv' 
file_ext = 'wav'

label_class = 'MW'  # label to use for the sound clip
label_subclass = '' # label to use for the sound clip subclass

In [39]:
annot = create_minke_annot(audio_dir, deployment_file, file_ext, label_class, label_subclass)

channelA_2020-10-19_00-00-04_minke_0.wav
channelA_2020-10-19_00-00-04_minke_1.wav
channelA_2020-10-19_00-00-04_minke_2.wav
channelA_2020-10-19_00-00-04_minke_3.wav
channelA_2020-10-19_01-00-04_minke_0.wav
channelA_2020-10-19_01-00-04_minke_1.wav
channelA_2020-10-19_01-00-04_minke_10.wav
channelA_2020-10-19_01-00-04_minke_11.wav
channelA_2020-10-19_01-00-04_minke_12.wav
channelA_2020-10-19_01-00-04_minke_13.wav
channelA_2020-10-19_01-00-04_minke_14.wav
channelA_2020-10-19_01-00-04_minke_15.wav
channelA_2020-10-19_01-00-04_minke_16.wav
channelA_2020-10-19_01-00-04_minke_17.wav
channelA_2020-10-19_01-00-04_minke_18.wav
channelA_2020-10-19_01-00-04_minke_19.wav
channelA_2020-10-19_01-00-04_minke_2.wav
channelA_2020-10-19_01-00-04_minke_20.wav
channelA_2020-10-19_01-00-04_minke_21.wav
channelA_2020-10-19_01-00-04_minke_22.wav
channelA_2020-10-19_01-00-04_minke_23.wav
channelA_2020-10-19_01-00-04_minke_24.wav
channelA_2020-10-19_01-00-04_minke_3.wav
channelA_2020-10-19_01-00-04_minke_4.wav
c

channelA_2020-11-07_06-00-04_minke_10.wav
channelA_2020-11-07_06-00-04_minke_11.wav
channelA_2020-11-07_06-00-04_minke_12.wav
channelA_2020-11-07_06-00-04_minke_13.wav
channelA_2020-11-07_06-00-04_minke_14.wav
channelA_2020-11-07_06-00-04_minke_15.wav
channelA_2020-11-07_06-00-04_minke_16.wav
channelA_2020-11-07_06-00-04_minke_17.wav
channelA_2020-11-07_06-00-04_minke_18.wav
channelA_2020-11-07_06-00-04_minke_19.wav
channelA_2020-11-07_06-00-04_minke_2.wav
channelA_2020-11-07_06-00-04_minke_20.wav
channelA_2020-11-07_06-00-04_minke_21.wav
channelA_2020-11-07_06-00-04_minke_22.wav
channelA_2020-11-07_06-00-04_minke_23.wav
channelA_2020-11-07_06-00-04_minke_24.wav
channelA_2020-11-07_06-00-04_minke_25.wav
channelA_2020-11-07_06-00-04_minke_26.wav
channelA_2020-11-07_06-00-04_minke_27.wav
channelA_2020-11-07_06-00-04_minke_28.wav
channelA_2020-11-07_06-00-04_minke_29.wav
channelA_2020-11-07_06-00-04_minke_3.wav
channelA_2020-11-07_06-00-04_minke_30.wav
channelA_2020-11-07_06-00-04_minke_3

channelA_2020-11-07_13-00-04_minke_17.wav
channelA_2020-11-07_13-00-04_minke_18.wav
channelA_2020-11-07_13-00-04_minke_19.wav
channelA_2020-11-07_13-00-04_minke_2.wav
channelA_2020-11-07_13-00-04_minke_20.wav
channelA_2020-11-07_13-00-04_minke_21.wav
channelA_2020-11-07_13-00-04_minke_22.wav
channelA_2020-11-07_13-00-04_minke_23.wav
channelA_2020-11-07_13-00-04_minke_24.wav
channelA_2020-11-07_13-00-04_minke_25.wav
channelA_2020-11-07_13-00-04_minke_26.wav
channelA_2020-11-07_13-00-04_minke_27.wav
channelA_2020-11-07_13-00-04_minke_28.wav
channelA_2020-11-07_13-00-04_minke_29.wav
channelA_2020-11-07_13-00-04_minke_3.wav
channelA_2020-11-07_13-00-04_minke_30.wav
channelA_2020-11-07_13-00-04_minke_31.wav
channelA_2020-11-07_13-00-04_minke_32.wav
channelA_2020-11-07_13-00-04_minke_33.wav
channelA_2020-11-07_13-00-04_minke_34.wav
channelA_2020-11-07_13-00-04_minke_35.wav
channelA_2020-11-07_13-00-04_minke_36.wav
channelA_2020-11-07_13-00-04_minke_37.wav
channelA_2020-11-07_13-00-04_minke_3

Let's look at the summary of annotations that were created:

In [40]:
annot.summary()

label_class,MW,Total
deployment_ID,,
UK-SAMS-WestScotland-202009-N1,563,563
Total,563,563


The dataset can now be saved as a Raven annotation file and netcdf4 file:

In [41]:
annot.to_netcdf(os.path.join(audio_dir, 'Annotations_dataset_' + annot.data['deployment_ID'][0] +' annotations.nc'))
annot.to_raven(audio_dir, outfile='Annotations_dataset_' + annot.data['deployment_ID'][0] +'.Table.1.selections.txt', single_file=True)

### Dataset 6: UK-SAMS-WestScotland-202009-N2

Definition of all the paths of all folders with the raw annotation and audio files for this deployment.

In [42]:
audio_dir = r'D:\NOAA\2022_Minke_whale_detector\manual_annotations\UK-SAMS-WestScotland-202009-N2'
deployment_file = r'deployment_info.csv' 
file_ext = 'wav'

label_class = 'MW'  # label to use for the sound clip
label_subclass = '' # label to use for the sound clip subclass

In [43]:
annot = create_minke_annot(audio_dir, deployment_file, file_ext, label_class, label_subclass)

channelA_2020-09-28_01-00-04_minke_0.wav
channelA_2020-09-28_01-00-04_minke_1.wav
channelA_2020-09-28_01-00-04_minke_2.wav
channelA_2020-09-28_01-00-04_minke_3.wav
channelA_2020-09-28_01-00-04_minke_4.wav
channelA_2020-09-28_01-00-04_minke_5.wav
channelA_2020-09-28_01-00-04_minke_6.wav
channelA_2020-09-28_01-00-04_minke_7.wav
channelA_2020-09-28_01-00-04_minke_8.wav
channelA_2020-09-28_03-00-03_minke_0.wav
channelA_2020-09-28_03-00-03_minke_1.wav
channelA_2020-09-28_03-00-03_minke_2.wav
channelA_2020-09-28_09-00-03_minke_0.wav
channelA_2020-09-28_09-00-03_minke_1.wav
channelA_2020-09-28_19-00-04_minke_0.wav
channelA_2020-09-28_23-00-04_minke_0.wav
channelA_2020-09-28_23-00-04_minke_1.wav
channelA_2020-09-28_23-00-04_minke_10.wav
channelA_2020-09-28_23-00-04_minke_11.wav
channelA_2020-09-28_23-00-04_minke_12.wav
channelA_2020-09-28_23-00-04_minke_13.wav
channelA_2020-09-28_23-00-04_minke_14.wav
channelA_2020-09-28_23-00-04_minke_15.wav
channelA_2020-09-28_23-00-04_minke_16.wav
channelA_

Let's look at the summary of annotations that were created:

In [44]:
annot.summary()

label_class,MW,Total
deployment_ID,,
UK-SAMS-WestScotland-202009-N2,104,104
Total,104,104


The dataset can now be saved as a Raven annotation file and netcdf4 file:

In [45]:
annot.to_netcdf(os.path.join(audio_dir, 'Annotations_dataset_' + annot.data['deployment_ID'][0] +' annotations.nc'))
annot.to_raven(audio_dir, outfile='Annotations_dataset_' + annot.data['deployment_ID'][0] +'.Table.1.selections.txt', single_file=True)

### Dataset 7: UK-SAMS-WestScotland-202010-EL1

Definition of all the paths of all folders with the raw annotation and audio files for this deployment.

In [46]:
audio_dir = r'D:\NOAA\2022_Minke_whale_detector\manual_annotations\UK-SAMS-WestScotland-202010-EL1'
deployment_file = r'deployment_info.csv' 
file_ext = 'wav'

label_class = 'MW'  # label to use for the sound clip
label_subclass = '' # label to use for the sound clip subclass

In [47]:
annot = create_minke_annot(audio_dir, deployment_file, file_ext, label_class, label_subclass)

channelA_2020-10-23_08-00-03_minke_0.wav
channelA_2020-11-17_00-00-04_minke_0.wav
channelA_2020-11-17_00-00-04_minke_1.wav
channelA_2020-11-17_00-00-04_minke_2.wav
channelA_2020-11-17_00-00-04_minke_3.wav
channelA_2020-11-17_00-00-04_minke_4.wav
channelA_2020-11-17_00-00-04_minke_5.wav
channelA_2020-11-17_01-00-04_minke_0.wav
channelA_2020-11-17_01-00-04_minke_1.wav
channelA_2020-11-17_01-00-04_minke_2.wav
channelA_2020-11-17_01-00-04_minke_3.wav
channelA_2020-11-17_01-00-04_minke_4.wav
channelA_2020-11-17_01-00-04_minke_5.wav
channelA_2020-11-17_02-00-04_minke_0.wav
channelA_2020-11-17_02-00-04_minke_1.wav
channelA_2020-11-17_02-00-04_minke_10.wav
channelA_2020-11-17_02-00-04_minke_2.wav
channelA_2020-11-17_02-00-04_minke_3.wav
channelA_2020-11-17_02-00-04_minke_4.wav
channelA_2020-11-17_02-00-04_minke_5.wav
channelA_2020-11-17_02-00-04_minke_6.wav
channelA_2020-11-17_02-00-04_minke_7.wav
channelA_2020-11-17_02-00-04_minke_8.wav
channelA_2020-11-17_02-00-04_minke_9.wav
channelA_2020-1

channelA_2020-11-17_20-00-04_minke_9.wav
channelA_2020-11-17_21-00-04_minke_0.wav
channelA_2020-11-17_21-00-04_minke_1.wav
channelA_2020-11-17_21-00-04_minke_10.wav
channelA_2020-11-17_21-00-04_minke_11.wav
channelA_2020-11-17_21-00-04_minke_12.wav
channelA_2020-11-17_21-00-04_minke_13.wav
channelA_2020-11-17_21-00-04_minke_14.wav
channelA_2020-11-17_21-00-04_minke_15.wav
channelA_2020-11-17_21-00-04_minke_16.wav
channelA_2020-11-17_21-00-04_minke_17.wav
channelA_2020-11-17_21-00-04_minke_18.wav
channelA_2020-11-17_21-00-04_minke_2.wav
channelA_2020-11-17_21-00-04_minke_3.wav
channelA_2020-11-17_21-00-04_minke_4.wav
channelA_2020-11-17_21-00-04_minke_5.wav
channelA_2020-11-17_21-00-04_minke_6.wav
channelA_2020-11-17_21-00-04_minke_7.wav
channelA_2020-11-17_21-00-04_minke_8.wav
channelA_2020-11-17_21-00-04_minke_9.wav
channelA_2020-11-17_22-00-04_minke_0.wav
channelA_2020-11-17_22-00-04_minke_1.wav
channelA_2020-11-17_22-00-04_minke_2.wav
channelA_2020-11-17_22-00-04_minke_3.wav
channel

Let's look at the summary of annotations that were created:

In [48]:
annot.summary()

label_class,MW,Total
deployment_ID,,
UK-SAMS-WestScotland-202010-EL1,238,238
Total,238,238


The dataset can now be saved as a Raven annotation file and netcdf4 file:

In [49]:
annot.to_netcdf(os.path.join(audio_dir, 'Annotations_dataset_' + annot.data['deployment_ID'][0] +' annotations.nc'))
annot.to_raven(audio_dir, outfile='Annotations_dataset_' + annot.data['deployment_ID'][0] +'.Table.1.selections.txt', single_file=True)

### Dataset 8: UK-SAMS-WestScotland-202010-S1

Definition of all the paths of all folders with the raw annotation and audio files for this deployment.

In [50]:
audio_dir = r'D:\NOAA\2022_Minke_whale_detector\manual_annotations\UK-SAMS-WestScotland-202010-S1'
deployment_file = r'deployment_info.csv' 
file_ext = 'wav'

label_class = 'MW'  # label to use for the sound clip
label_subclass = '' # label to use for the sound clip subclass

In [51]:
annot = create_minke_annot(audio_dir, deployment_file, file_ext, label_class, label_subclass)

channelA_2021-04-28_07-00-05_minke_0.wav
channelA_2021-04-28_07-00-05_minke_1.wav
channelA_2021-05-20_00-00-05_minke_0.wav
channelA_2021-08-06_23-00-04_minke_0.wav


Let's look at the summary of annotations that were created:

In [52]:
annot.summary()

label_class,MW,Total
deployment_ID,,
UK-SAMS-WestScotland-202010-S1,4,4
Total,4,4


The dataset can now be saved as a Raven annotation file and netcdf4 file:

In [53]:
annot.to_netcdf(os.path.join(audio_dir, 'Annotations_dataset_' + annot.data['deployment_ID'][0] +' annotations.nc'))
annot.to_raven(audio_dir, outfile='Annotations_dataset_' + annot.data['deployment_ID'][0] +'.Table.1.selections.txt', single_file=True)

### Dataset 9: UK-UAberdeen-MorayFirth-201905_987-112

Definition of all the paths of all folders with the raw annotation and audio files for this deployment.

In [54]:
audio_dir = r'D:\NOAA\2022_Minke_whale_detector\manual_annotations\UK-UAberdeen-MorayFirth-201905_987-112'
deployment_file = r'deployment_info.csv' 
file_ext = 'wav'

label_class = 'MW'  # label to use for the sound clip
label_subclass = '' # label to use for the sound clip subclass

In [59]:
annot = create_minke_annot(audio_dir, deployment_file, file_ext, label_class, label_subclass)

Depl987_1347451_190611004441_minke_1_7.wav
Depl987_1347451_190611001441_minke_1_0.wav
Depl987_1347451_190611001441_minke_1_1.wav
Depl987_1347451_190611001441_minke_1_2.wav
Depl987_1347451_190611001441_minke_2_3.wav
Depl987_1347451_190611004441_minke_1_0.wav
Depl987_1347451_190611004441_minke_1_1.wav
Depl987_1347451_190611004441_minke_1_2.wav
Depl987_1347451_190611004441_minke_1_3.wav
Depl987_1347451_190611004441_minke_1_4.wav
Depl987_1347451_190611004441_minke_1_5.wav
Depl987_1347451_190611004441_minke_1_6.wav


Let's look at the summary of annotations that were created:

In [60]:
annot.summary()

label_class,MW,Total
deployment_ID,,
UK-UAberdeen-MorayFirth-201905_987-112,12,12
Total,12,12


The dataset can now be saved as a Raven annotation file and netcdf4 file:

In [61]:
annot.to_netcdf(os.path.join(audio_dir, 'Annotations_dataset_' + annot.data['deployment_ID'][0] +' annotations.nc'))
annot.to_raven(audio_dir, outfile='Annotations_dataset_' + annot.data['deployment_ID'][0] +'.Table.1.selections.txt', single_file=True)

### Dataset 10: UK-UAberdeen-MorayFirth-202006_1101-164

Definition of all the paths of all folders with the raw annotation and audio files for this deployment.

In [62]:
audio_dir = r'D:\NOAA\2022_Minke_whale_detector\manual_annotations\UK-UAberdeen-MorayFirth-202006_1101-164'
deployment_file = r'deployment_info.csv' 
file_ext = 'wav'

label_class = 'MW'  # label to use for the sound clip
label_subclass = '' # label to use for the sound clip subclass

In [63]:
annot = create_minke_annot(audio_dir, deployment_file, file_ext, label_class, label_subclass)

Depl1101_134541352_200613170002_minke_2_0.wav
Depl1101_134541352_200607060002_minke_1_0.wav
Depl1101_134541352_200607060002_minke_1_1.wav
Depl1101_134541352_200607060002_minke_1_3.wav
Depl1101_134541352_200607060002_minke_1_4.wav
Depl1101_134541352_200607060002_minke_2_2.wav


Let's look at the summary of annotations that were created:

In [64]:
annot.summary()

label_class,MW,Total
deployment_ID,,
UK-UAberdeen-MorayFirth-202006_1101-164,6,6
Total,6,6


The dataset can now be saved as a Raven annotation file and netcdf4 file:

In [65]:
annot.to_netcdf(os.path.join(audio_dir, 'Annotations_dataset_' + annot.data['deployment_ID'][0] +' annotations.nc'))
annot.to_raven(audio_dir, outfile='Annotations_dataset_' + annot.data['deployment_ID'][0] +'.Table.1.selections.txt', single_file=True)

### Dataset 11: UK-UAberdeen-MorayFirth-202006_1102-112

Definition of all the paths of all folders with the raw annotation and audio files for this deployment.

In [66]:
audio_dir = r'D:\NOAA\2022_Minke_whale_detector\manual_annotations\UK-UAberdeen-MorayFirth-202006_1102-112'
deployment_file = r'deployment_info.csv' 
file_ext = 'wav'

label_class = 'MW'  # label to use for the sound clip
label_subclass = '' # label to use for the sound clip subclass

In [67]:
annot = create_minke_annot(audio_dir, deployment_file, file_ext, label_class, label_subclass)

Depl1102_1678254119_200607040002_minke_2_0.wav


Let's look at the summary of annotations that were created:

In [68]:
annot.summary()

label_class,MW,Total
deployment_ID,,
UK-UAberdeen-MorayFirth-202006_1102-112,1,1
Total,1,1


The dataset can now be saved as a Raven annotation file and netcdf4 file:

In [69]:
annot.to_netcdf(os.path.join(audio_dir, 'Annotations_dataset_' + annot.data['deployment_ID'][0] +' annotations.nc'))
annot.to_raven(audio_dir, outfile='Annotations_dataset_' + annot.data['deployment_ID'][0] +'.Table.1.selections.txt', single_file=True)

### Dataset 12: UK-UAberdeen-MorayFirth-202105_1160-112

Definition of all the paths of all folders with the raw annotation and audio files for this deployment.

In [70]:
audio_dir = r'D:\NOAA\2022_Minke_whale_detector\manual_annotations\UK-UAberdeen-MorayFirth-202105_1160-112'
deployment_file = r'deployment_info.csv' 
file_ext = 'wav'

label_class = 'MW'  # label to use for the sound clip
label_subclass = '' # label to use for the sound clip subclass

In [71]:
annot = create_minke_annot(audio_dir, deployment_file, file_ext, label_class, label_subclass)

Depl1160_1678036995_210902010002_minke_1_0.wav
Depl1160_1678036995_210902000002_minke_1_0.wav
Depl1160_1678036995_210902000002_minke_1_1.wav


Let's look at the summary of annotations that were created:

In [72]:
annot.summary()

label_class,MW,Total
deployment_ID,,
UK-UAberdeen-MorayFirth-202105_1160-112,3,3
Total,3,3


The dataset can now be saved as a Raven annotation file and netcdf4 file:

In [73]:
annot.to_netcdf(os.path.join(audio_dir, 'Annotations_dataset_' + annot.data['deployment_ID'][0] +' annotations.nc'))
annot.to_raven(audio_dir, outfile='Annotations_dataset_' + annot.data['deployment_ID'][0] +'.Table.1.selections.txt', single_file=True)

### Dataset 13: UK-UAberdeen-MorayFirth-202107_1173-281

Definition of all the paths of all folders with the raw annotation and audio files for this deployment.

In [74]:
audio_dir = r'D:\NOAA\2022_Minke_whale_detector\manual_annotations\UK-UAberdeen-MorayFirth-202107_1173-281'
deployment_file = r'deployment_info.csv' 
file_ext = 'wav'

label_class = 'MW'  # label to use for the sound clip
label_subclass = '' # label to use for the sound clip subclass

In [75]:
annot = create_minke_annot(audio_dir, deployment_file, file_ext, label_class, label_subclass)

Depl1173_1678508072_210714010002_minke_1_20.wav
Depl1173_1678508072_210704090002_minke_1_0.wav
Depl1173_1678508072_210704090002_minke_1_1.wav
Depl1173_1678508072_210704090002_minke_1_2.wav
Depl1173_1678508072_210704090002_minke_1_3.wav
Depl1173_1678508072_210705110002_minke_1_0.wav
Depl1173_1678508072_210705110002_minke_1_1.wav
Depl1173_1678508072_210705110002_minke_1_2.wav
Depl1173_1678508072_210705110002_minke_1_3.wav
Depl1173_1678508072_210708080002_minke_1_0.wav
Depl1173_1678508072_210708080002_minke_1_1.wav
Depl1173_1678508072_210708080002_minke_1_3.wav
Depl1173_1678508072_210708080002_minke_2_2.wav
Depl1173_1678508072_210708200002_minke_1_0.wav
Depl1173_1678508072_210708200002_minke_1_1.wav
Depl1173_1678508072_210708200002_minke_1_2.wav
Depl1173_1678508072_210710010002_minke_1_0.wav
Depl1173_1678508072_210710010002_minke_1_1.wav
Depl1173_1678508072_210710010002_minke_1_2.wav
Depl1173_1678508072_210710010002_minke_2_3.wav
Depl1173_1678508072_210710090002_minke_1_0.wav
Depl1173_167

Depl1173_1678508072_210714010002_minke_1_2.wav
Depl1173_1678508072_210714010002_minke_1_3.wav
Depl1173_1678508072_210714010002_minke_1_4.wav
Depl1173_1678508072_210714010002_minke_1_5.wav
Depl1173_1678508072_210714010002_minke_1_6.wav
Depl1173_1678508072_210714010002_minke_1_7.wav
Depl1173_1678508072_210714010002_minke_1_8.wav
Depl1173_1678508072_210714010002_minke_1_9.wav
Depl1173_1678508072_210714010002_minke_1_10.wav
Depl1173_1678508072_210714010002_minke_1_11.wav
Depl1173_1678508072_210714010002_minke_1_12.wav
Depl1173_1678508072_210714010002_minke_1_13.wav
Depl1173_1678508072_210714010002_minke_1_14.wav
Depl1173_1678508072_210714010002_minke_1_15.wav
Depl1173_1678508072_210714010002_minke_1_16.wav
Depl1173_1678508072_210714010002_minke_1_17.wav
Depl1173_1678508072_210714010002_minke_1_18.wav
Depl1173_1678508072_210714010002_minke_1_19.wav


Let's look at the summary of annotations that were created:

In [76]:
annot.summary()

label_class,MW,Total
deployment_ID,,
UK-UAberdeen-MorayFirth-202107_1173-281,195,195
Total,195,195


The dataset can now be saved as a Raven annotation file and netcdf4 file:

In [77]:
annot.to_netcdf(os.path.join(audio_dir, 'Annotations_dataset_' + annot.data['deployment_ID'][0] +' annotations.nc'))
annot.to_raven(audio_dir, outfile='Annotations_dataset_' + annot.data['deployment_ID'][0] +'.Table.1.selections.txt', single_file=True)

### Dataset 14: UK-UAberdeen-MorayFirth-202107_1190-295

Definition of all the paths of all folders with the raw annotation and audio files for this deployment.

In [78]:
audio_dir = r'D:\NOAA\2022_Minke_whale_detector\manual_annotations\UK-UAberdeen-MorayFirth-202107_1190-295'
deployment_file = r'deployment_info.csv' 
file_ext = 'wav'

label_class = 'MW'  # label to use for the sound clip
label_subclass = '' # label to use for the sound clip subclass

In [79]:
annot = create_minke_annot(audio_dir, deployment_file, file_ext, label_class, label_subclass)

Depl1190_6281_210714175733_minke_2_6.wav
Depl1190_6281_210713175740_minke_1_0.wav
Depl1190_6281_210713175740_minke_1_1.wav
Depl1190_6281_210713175740_minke_1_10.wav
Depl1190_6281_210713175740_minke_1_11.wav
Depl1190_6281_210713175740_minke_2_2.wav
Depl1190_6281_210713175740_minke_2_3.wav
Depl1190_6281_210713175740_minke_2_4.wav
Depl1190_6281_210713175740_minke_2_5.wav
Depl1190_6281_210713175740_minke_2_6.wav
Depl1190_6281_210713175740_minke_2_7.wav
Depl1190_6281_210713175740_minke_2_8.wav
Depl1190_6281_210713175740_minke_2_9.wav
Depl1190_6281_210713205739_minke_1_0.wav
Depl1190_6281_210713205739_minke_1_1.wav
Depl1190_6281_210713205739_minke_1_2.wav
Depl1190_6281_210713205739_minke_1_3.wav
Depl1190_6281_210713205739_minke_1_4.wav
Depl1190_6281_210713205739_minke_1_5.wav
Depl1190_6281_210713205739_minke_1_6.wav
Depl1190_6281_210713205739_minke_1_7.wav
Depl1190_6281_210713205739_minke_1_8.wav
Depl1190_6281_210713205739_minke_1_11.wav
Depl1190_6281_210713205739_minke_1_12.wav
Depl1190_628

Let's look at the summary of annotations that were created:

In [80]:
annot.summary()

label_class,MW,Total
deployment_ID,,
UK-UAberdeen-MorayFirth-202107_1190-295,73,73
Total,73,73


The dataset can now be saved as a Raven annotation file and netcdf4 file:

In [81]:
annot.to_netcdf(os.path.join(audio_dir, 'Annotations_dataset_' + annot.data['deployment_ID'][0] +' annotations.nc'))
annot.to_raven(audio_dir, outfile='Annotations_dataset_' + annot.data['deployment_ID'][0] +'.Table.1.selections.txt', single_file=True)